# Transformer Encoder-Decoder Walk-Through  
**Translating “Hello World” → “Bonjour le monde”**

<small>Illustrates how the encoder builds contextual representations and how the decoder turns them into French tokens.</small>

## 1  Install & import
We’ll use Hugging Face 🤗 *transformers* and the pre‑trained MarianMT model (`Helsinki‑NLP/opus-mt-en-fr`).

In [ ]:
!pip -q install transformers sentencepiece --upgrade
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch, numpy as np

## 2  Load tokenizer & model
This model has a **shared vocabulary** for both languages but a classic encoder‑decoder architecture internally.

In [ ]:
model_name = "Helsinki-NLP/opus-mt-en-fr"
tokenizer  = AutoTokenizer.from_pretrained(model_name)
model      = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model.eval();  # inference mode

## 3  Encode the English sentence
`tokenizer` maps the string → IDs.  
The **encoder** processes those IDs into a matrix of contextual hidden states (one vector per input token).

In [ ]:
sentence_en = "Hello World"
inputs = tokenizer(sentence_en, return_tensors="pt")
print("Input IDs:", inputs["input_ids"][0].tolist(),
      "\nTokens   :", tokenizer.convert_ids_to_tokens(inputs["input_ids"][0]))

with torch.no_grad():
    encoder_outputs = model.get_encoder()(
        input_ids      = inputs["input_ids"],
        attention_mask = inputs["attention_mask"],
        return_dict    = True
    )

hidden_states = encoder_outputs.last_hidden_state           # shape: [batch, seq_len, hidden_dim]
print("Encoder hidden-state shape :", hidden_states.shape)

### What did we just get?
* Each **token** (“▁Hello”, “▁World”) now has a 512‑dimensional vector.  
* Self‑attention inside the encoder lets every token *see* every other token, so
  each vector is **contextual** (“Hello” knows “World” follows it).

## 4  Decode step‑by‑step (greedy) – behind the curtain
We’ll feed the encoder’s representations to the **decoder** one token at a time
so you can watch how it auto‑regressively builds the French sentence.

In [ ]:
# Helper to convert IDs → readable token string
def pretty(ids):
    return tokenizer.convert_ids_to_tokens(ids, skip_special_tokens=True)

enc_out = encoder_outputs.last_hidden_state

decoded_ids = [tokenizer.lang_code_to_id["fr"]] if hasattr(tokenizer, "lang_code_to_id") else []
for step in range(10):                         # limit to 10 steps for safety
    decoder_input = torch.tensor([decoded_ids]).to(model.device)
    with torch.no_grad():
        outputs = model(
            encoder_outputs = (enc_out,),
            decoder_input_ids = decoder_input,
            return_dict = True
        )
    next_id = int(outputs.logits[0, -1].argmax())  # greedy
    decoded_ids.append(next_id)
    if next_id == tokenizer.eos_token_id:
        break
    print(f"Step {step+1}: {pretty(decoded_ids)}")

## 5  Full translation with `.generate()`
Most of the time you’ll just call `model.generate()`; it will handle beam search, length penalties, etc.

In [ ]:
translated_ids = model.generate(**inputs, max_length=40)
translation_fr = tokenizer.decode(translated_ids[0], skip_special_tokens=True)
print("🔤 English :", sentence_en)
print("🇫🇷 French  :", translation_fr)

## 6  Recap – Encoder vs Decoder

| Stage | What it *sees* | Learns / produces | Why it matters |
|-------|----------------|-------------------|----------------|
| **Encoder** | Entire source sentence *simultaneously* via self‑attention | Context‑rich hidden vectors `h₁ … hₙ` | Captures meaning & word interactions in the *source* language |
| **Decoder** | Its own past outputs **plus** the encoder vectors (via encoder‑decoder attention) | Next‑token distribution \(P(y_t \| y_{<t}, h)\) | Generates target language tokens one‑by‑one, staying fluent & faithful |

* Auto‑regressive = *predict next token given everything so far* (teacher forcing when training, greedy/beam search when decoding).
* No separate “language model” is needed – the decoder **is** a conditional language model conditioned on the encoder.

## 7  Playground ideas for students
1. **Change the sentence** to test out longer inputs or idioms (`"How are you?"` → `"Comment ça va ?"`).
2. Compare **beam search vs greedy**: `model.generate(..., num_beams=5)`.
3. Inspect **attention weights**: set `output_attentions=True` when calling `model(...)`.
4. Replace MarianMT with **T5** (`t5-small`) and prepend the task prefix `"translate English to French: "` to the input.